# Tensorflow - Marker Quality Control

In [ ]:
import mlflow
import tensorflow as tf

### Chargement du modèle

In [ ]:
model = tf.saved_model.load(
    "marker_quality_control/1", tags=None, options=None
)

## Prédiction sur une image

In [ ]:
img_path = "images/10.jpg"
img = tf.keras.utils.load_img(img_path, target_size=(224, 224))
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, axis=0)
img_array = img_array / 255.0

In [ ]:
model(inputs = img_array)

### Encapsulation dans un mlflow pyfunc modèle


In [ ]:
import mlflow

mlflow.set_tracking_uri("http://model-platform.com/registry/test/")
mlflow.pyfunc.log_model(
            artifact_path="custom_model",
            python_model="marker_quality_control_wrapper.py",
            artifacts={"model_path": "marker_quality_control/1"},
            registered_model_name="marker_quality_control"
        )


## Prédiction en pullant le modèle depuis la modèle plateform

In [ ]:
model = mlflow.pyfunc.load_model(model_uri="models:/marker_quality_control/1")

In [ ]:
model.predict(img)

### Faire un curl sur l'endpoint déployé avec une image
curl -X POST http://model-platform.com/deploy/test/test-marker-quality-control-1-deployment-3b4041/predict \
  -F "file=@images/10.jpg"